In [1]:
import TimeTagger
import numpy as np

## channel/channel example (photon mean per channel)

In [2]:
class PhotonMean(TimeTagger.CustomMeasurement):
    """
    CustomMeasurement that gets channel_1/channel_2.
    """

    def __init__(self, tagger, channels):
        TimeTagger.CustomMeasurement.__init__(self, tagger)
        self.channels = list(channels)
        self.counts = np.zeros((len(channels),), dtype=np.int32)

        # Each used channel must be registered
        for channel in channels:
            self.register_channel(channel)

        self.clear_impl()

        # At the end of a Measurement construction we must indicate that we
        # have finished
        self.finalize_init()

    def __del__(self):
        # The measurement must be stopped before deconstruction to avoid
        # concurrent measure() calls
        self.stop()

    def getData(self):
        # lock this instance to avoid conflicting results while measure is
        # running apart.
        with self.mutex:
            counts = np.array(self.counts)
        return counts[0]/counts[1]

    def getIndex(self):
        # this method does not depend on the internal state, so there is no
        # need for a lock
        return list(self.channels)

    def clear_impl(self):
        # the lock is already acquired
        self.counts *= 0

    def on_start(self):
        # the lock is already acquired
        pass

    def on_stop(self):
        # the lock is already acquired
        pass

    def process(self, incoming_tags, begin_time, end_time):
        # the lock is already acquired
        # self.data is provided as reference, so it must not be accessed
        # anywhere else without locking the mutex.
        # incoming_tags is provided as a read-only reference. The storage will
        # be deallocated after this call, so you must not store a reference to
        # this object. Make a copy instead.
        channel_numbers, counts = np.unique(incoming_tags["channel"], return_counts=True)
        for channel_number, count in zip(channel_numbers, counts):
            self.counts[self.channels.index(channel_number)] += count
            
            if count%1E5:
                print(count)

In [3]:
photon = 4
frame = 3

In [4]:
dump_file = r"Z:\PriyaM\swabian_data\20230712_tissue\test_tissue_2023-07-12_154752.1.ttbin"

In [5]:
tagger = TimeTagger.createTimeTaggerVirtual()

In [6]:
photon_mean = PhotonMean(tagger, channels=[photon, frame])

In [7]:
tagger.getConfiguration()

{'configure version': 1,
 'current time': '2023-07-18 13:50:48 -0500',
 'hardware configuration': None,
 'measurements': [{'name': 'CustomMeasurement',
   'params': None,
   'registered channels': [3, 4],
   'virtual channels': []}],
 'replay speed': -1.0,
 'software clock': None,
 'software version': '2.16.2',
 'virtual inputs': [{'channel': 3,
   'conditional filter filtered': False,
   'conditional filter triggers': False,
   'deadtime': 0,
   'delay': 0,
   'testsignal': False},
  {'channel': 4,
   'conditional filter filtered': False,
   'conditional filter triggers': False,
   'deadtime': 0,
   'delay': 0,
   'testsignal': False}]}

In [8]:
tagger.setReplaySpeed(-1)
replay = tagger.replay(dump_file)
tagger.waitForCompletion()

10
8
8
6
4
4
3
6
5
6
8
7
4
3
2
5
1
6
4
8
6
6
4
8
4
4
7
5
5
8
5
3
8
5
6
6
5
5
2
2
6
10
6
4
5
7
5
10
4
3
7
5
2
5
11
2
5
5
3
7
5
7
7
5
1
6
5
10
6
4
7
4
4
6
3
3
5
2
6
4
6
5
7
5
2
2
2
3
4
4
9
3
3
3
3
6
5
3
4
10
1
4
10
9
3
6
6
2
6
6
8
6
8
6
3
2
7
5
4
7
4
6
6
9
6
4
8
8
2
4
5
6
2
2
5
5
2
6
10
6
2
9
3
6
2
5
1
2
4
8
6
5
4
12
4
6
9
5
6
4
4
6
5
7
6
10
1
5
9
9
5
7
3
5
8
9
7
8
3
3
4
6
3
5
5
9
3
5
5
6
7
6
4
2
6
5
6
6
5
4
3
1
10
3
4
6
4
6
2
7
8
4
8
10
4
4
11
5
3
5
5
7
4
7
6
6
2
2
7
8
7
5
5
6
2
5
4
4
7
4
4
3
8
4
2
5
4
6
4
5
5
9
5
3
3
4
5
3
4
2
7
5
2
2
4
4
3
5
7
5
3
8
5
4
6
9
4
5
6
2
5
10
6
6
8
6
5
3
4
2
2
5
5
2
4
1
7
4
5
4
6
3
6
3
9
4
1
8
2
3
6
8
3
8
5
8
8
4
5
7
2
7
3
8
4
4
6
6
2
2
3
8
6
5
6
7
6
5
3
6
5
5
7
2
5
3
3
3
6
3
7
4
6
4
5
9
5
3
3
4
6
2
5
6
9
5
7
7
1
3
5
2
7
11
6
5
4
5
8
4
1
6
7
7
6
6
3
7
663
3221
11326
24608
43010
43027
42546
37034
38088
38977
39372
39607
39396
39276
38961
38546
37678
32156
19030
16702
1
38307
39802
39621
37269
29316
38938
28545
20461
21122
39394
39319
39014
38638
37943
36172


True

In [9]:
photon_mean_data = photon_mean.getData()
print("mean photons per frame:", photon_mean_data)


mean photons per frame: 622230.6666666666


In [9]:
photon_mean.clear_impl()